<a href="https://colab.research.google.com/github/TonThatRon/RAG-with-Gemini-Faiss/blob/main/rag_with_gemini_faiss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG với FAISS và Gemini
---------------------------

Đây là một bài thực hành cơ bản về việc áp dụng mô hình RAG để tăng độ chính xác của thông tin từ nguồn dữ liệu được kiểm soát và giới
hạn ở __vector database__ (ở đây tôi sử dụng `FAISS`).
Sau đó sử dụng gemini như một chatbot để tương tác với dữ liệu và người dùng.
Sử dụng kỹ thuật prompt engineering là `few-shot prompting` >> nghĩa là hãy đưa vào cho chatbot model nhiều hơn 1 ngữ cảnh (dữ liệu hoăc/và gợi ý)
và yêu cầu chatbot chỉ sử dụng dữ liệu đã cung cấp làm nguồn thông tin để trả lời.

Ở đây chúng ta sử dụng **sentence-transformers** và pre-trained model:

**all-MiniML-L6-v2** từ __HuggingFace__

---------------------------

In [ ]:
#Cài đặt các package cần thiết
!pip install transformers #bạn có thể bỏ phần này đi. Tôi để đây để thử với một số model mà không dùng sentence transformer
!pip install numpy
!pip install pandas
!pip install sentence-transformers
!pip install google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.4/266.4 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.64.1
    Uninstalling grpcio-1.64.1:
      Successfully 

In [ ]:
import transformers
import pandas as pd
import numpy as np
import os

In [ ]:
# load dataset
# dataset là một file csv có hơn 100 địa điểm du lịch nổi tiếng ở Việt Nam được tôi tổng hợp ngắn gọn.
# bạn hoàn toàn có thể tạo một dataset riêng theo cách của bạn và sử dụng lại phần encoding phía dưới để tạo vector
# và insert vào faiss. Sau đó trải nghiệm với phần search và gemini.
dataset = os.path.join( "vn_landmark.csv")
df = pd.read_csv(dataset)
df.columns

Index(['name', 'description', 'province'], dtype='object')

**Sử dụng faiss để lưu trữ data**


In [ ]:
!pip install  faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.1 MB/s eta 0:00:00


In [ ]:
#encode data and add to vector database qdrant
import faiss
from sentence_transformers import SentenceTransformer
import csv

# Tải mô hình all-MiniLM-L6-v2
model = SentenceTransformer(
    "all-MiniLM-L6-v2", device="cuda" #nếu trên máy tính cá nhân thì để CPU và trên máy có GPU hoặc collab thì để GPU
)



# Encode data
texts = ['Tên địa danh/khu du lịch: ' + row.name + ', Mô tả:' + row.description + ', Địa phương/Tỉnh:' + row.province for row in df.itertuples()]
vectors = model.encode(texts, show_progress_bar=True)
# Initialize FAISS index (using IndexFlatIP for cosine similarity)
dimension = vectors.shape[1]
index = faiss.IndexFlatIP(dimension)
faiss.normalize_L2(vectors)  # Normalize vectors for cosine similarity
index.add(vectors.astype('float32'))
# payload sẽ được tạo bằng cách đọc trực tiếp file csv và sau đó lặp chuyển nó thành list và gán cho payload.
payload=[]
with open(dataset, "r") as fd:
    reader = csv.DictReader(fd)
    payloads = list(reader)


Batches:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
def search(text: str, encoder, faiss_index, payloads, k=3, threshold=0.2):
    # Convert text query into vector
    query_vector = encoder.encode([text]).astype('float32')
    faiss.normalize_L2(query_vector)

    # Search for closest vectors
    similarities, indices = faiss_index.search(query_vector, k)

    # Filter results based on similarity threshold
    filtered_results = [(payloads[i], sim) for i, sim in zip(indices[0], similarities[0]) if sim > threshold]

    return filtered_results

## Hãy tạo API key của bạn với Google AI Studio:
* Truy cập vào đường link này https://aistudio.google.com/app/prompts/new_chat hoặc https://ai.google.dev/aistudio
* Sau đó hãy login bằng acc gmail của bạn
* Sau đó chọn Get API key >> và chọn tiếp Create API Key nếu bạn chưa tạo 1 key nào trước đây hoặc đã xoá nó

In [ ]:
import google.generativeai as genai
from IPython.display import display, Markdown
#Hãy tạo gemini key thông qua Google AI Stuido trên Google Cloud Web sau đó bạn copy api key vào đây.
gemini_api_key="[you api key]"
genai.configure(api_key=gemini_api_key)
gemini_model = genai.GenerativeModel('gemini-1.5-flash')

In [ ]:
query = "Vịnh Hạ Long"
search_results = search(query, model, index, payloads)
search_results

[({'name': 'Vịnh Hạ Long',
   'description': 'Vịnh biển nằm ở phía đông bắc Việt Nam, thuộc vùng bờ biển tỉnh Quảng Ninh. Cách Hà Nội khoảng 165km về phía đông bắc. Nổi tiếng với hàng nghìn hòn đảo đá vôi và hang động tuyệt đẹp.',
   'province': 'Quảng Ninh'},
  0.46103057),
 ({'name': 'Hồ Tà Đùng',
   'description': "Nằm ở huyện Đắk G'long, cách thành phố Gia Nghĩa khoảng 40km về phía đông nam. Được mệnh danh là 'Vịnh Hạ Long của Tây Nguyên'.",
   'province': 'Đắk Nông'},
  0.45596185),
 ({'name': 'Hồ Tà Đùng',
   'description': "Nằm ở huyện Đắk G'long, cách thành phố Gia Nghĩa khoảng 40km về phía đông nam. Được mệnh danh là 'Vịnh Hạ Long của Tây Nguyên'.",
   'province': 'Đắk Nông'},
  0.45596185)]

In [ ]:
if search_results:
    context = "\n".join([f"Result {i+1} (similarity: {sim:.2f}): {res}" for i, (res, sim) in enumerate(search_results)])
    prompt = f'''prompt: {query}
You are an expert in landmark guidance. Use the following search results as context for your answer:

{context}

Please provide information about the landmark, focusing on the most relevant results. If the context is not relevant to the prompt, state that you don't have enough relevant information to answer.

Suggest some relevant questions based on the provided information. Always use Vietnamese in your response. Please add some web sources for reference if available.'''
else:
    prompt = f'prompt: {query}\nTôi không có đủ thông tin liên quan để trả lời câu hỏi về "{query}". Bạn có thể hỏi về một địa danh khác không?'

response = gemini_model.generate_content(prompt)
display(Markdown(response.text))

## Vịnh Hạ Long

Vịnh Hạ Long là một vịnh biển nằm ở phía đông bắc Việt Nam, thuộc vùng bờ biển tỉnh Quảng Ninh. Cách Hà Nội khoảng 165km về phía đông bắc. Vịnh Hạ Long nổi tiếng với hàng nghìn hòn đảo đá vôi và hang động tuyệt đẹp. 

**Câu hỏi liên quan:**

* Vịnh Hạ Long có gì đặc biệt?
* Những địa điểm nổi tiếng ở Vịnh Hạ Long là gì?
* Cách di chuyển đến Vịnh Hạ Long?
* Nên du lịch Vịnh Hạ Long vào mùa nào?

**Tham khảo thêm:**

* [Wikipedia](https://vi.wikipedia.org/wiki/V%E1%BB%89nh_H%E1%BA%A1_Long)
* [Du lịch Việt Nam](https://dulichvietnam.com.vn/vinh-ha-long/) 


In [ ]:
# Function to interactively search and get responses
def interactive_search():
    while True:
        query = input("Nhập câu hỏi của bạn (hoặc 'quit' để thoát): ")
        if query.lower() == 'quit':
            break

        search_results = search(query, model, index, payloads)

        if search_results:
            context = "\n".join([f"Result {i+1} (similarity: {sim:.2f}): {res}" for i, (res, sim) in enumerate(search_results)])
            prompt = f'''prompt: {query}
You are an expert in landmark guidance. Use the following search results as context for your answer:

{context}

Please provide information about the landmark, focusing on the most relevant results. If the context is not relevant to the prompt, state that you don't have enough relevant information to answer.

Suggest some relevant questions based on the provided information. Always use Vietnamese in your response. Please add some web sources for reference if available.'''
        else:
            prompt = f'prompt: {query}\nTôi không có đủ thông tin liên quan để trả lời câu hỏi về "{query}". Bạn có thể hỏi về một địa danh khác không?'

        response = gemini_model.generate_content(prompt)
        display(Markdown(response.text))
        print("\n")

# Run interactive search
interactive_search()

Nhập câu hỏi của bạn (hoặc 'quit' để thoát): Thác Bảo Đại


## Thác Bảo Đại: Một Điểm Du Lịch Hấp Dẫn Gần Đà Lạt

Thác Bảo Đại là một thác nước đẹp nằm ở xã Đạ Nhim, huyện Lạc Dương, tỉnh Lâm Đồng. Thác cách thành phố Đà Lạt khoảng 15km về phía đông bắc. 

Thác Bảo Đại được biết đến với vẻ đẹp tự nhiên hoang sơ, dòng nước trắng xóa đổ xuống từ độ cao tạo nên khung cảnh hùng vĩ.  Nơi đây còn gắn liền với truyền thuyết về vua Bảo Đại, vị vua cuối cùng của triều Nguyễn. 

**Một số câu hỏi liên quan:**

* Thác Bảo Đại có gì đặc biệt?
* Truyền thuyết về vua Bảo Đại và thác nước như thế nào?
* Cách di chuyển đến thác Bảo Đại?
* Có những hoạt động gì có thể trải nghiệm tại thác?
* Lưu trú gần thác Bảo Đại?

**Lưu ý:**  Thông tin chi tiết về thác Bảo Đại có thể được tìm thấy trên các trang web du lịch như:

* [Du lịch Lâm Đồng](https://dulichlamdong.vn/)
* [Vietnamtourism](https://www.vietnamtourism.com/)

Hãy khám phá vẻ đẹp hoang sơ của Thác Bảo Đại và tận hưởng một ngày thư giãn giữa thiên nhiên hùng vĩ của Lâm Đồng!




Nhập câu hỏi của bạn (hoặc 'quit' để thoát): Thác Bản Giốc


## Thác Bản Giốc

Thác Bản Giốc là một thác nước đẹp nằm trên biên giới Việt-Trung, tọa lạc tại xã Đàm Thủy, huyện Trùng Khánh, tỉnh Cao Bằng. Thác cách thành phố Cao Bằng khoảng 90km. 

**Một số câu hỏi liên quan:**

* Thác Bản Giốc có gì đặc biệt? 
* Làm sao để đến Thác Bản Giốc?
* Có những hoạt động gì có thể làm tại Thác Bản Giốc?
* Thác Bản Giốc có gì đẹp?
* Có những điểm tham quan nào khác gần Thác Bản Giốc?

**Nguồn tham khảo:**

* [Wikipedia](https://vi.wikipedia.org/wiki/Th%C3%A1c_B%E1%BA%A3n_Gi%E1%BB%91c)
* [Du lịch Cao Bằng](https://dulichcaobang.vn/dia-diem-du-lich/thac-ban-gioc)

Lưu ý: Thông tin được cung cấp dựa trên các kết quả tìm kiếm. Để biết thêm thông tin chi tiết, bạn có thể truy cập các trang web liên quan.




Nhập câu hỏi của bạn (hoặc 'quit' để thoát): quit
